### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline


# Read in the MovieTweetings dataset originally taken from https://github.com/sidooms/MovieTweetings/tree/master/latest
movies = pd.read_csv('movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [2]:
movies.head()

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [3]:
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0114508,8,1381006850
1,2,0208092,5,1586466072
2,2,0358273,9,1579057827
3,2,10039344,5,1578603053
4,2,6751668,9,1578955697


In [4]:
movies.shape[0]

35479

In [5]:
reviews.shape[0]

863866

In [6]:
# Josh testing the split() method on one item
movies.genre[0].split("|")

['Documentary', 'Short']

In [7]:
# we want to unpack genres column in movies dataframe to get the number of different genres
genres = []

for val in movies['genre']:
    try:
        genres.extend(val.split("|")) # Here Josh is using extend() - adding only elements instead of append() adding lists
        
    except AttributeError:
        pass
genres = set(genres) # transforming the list to a set you get only unique values
print(len(genres))
genres

28


{'Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Film-Noir',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Short',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western'}

In [8]:
len(np.unique(reviews.user_id))

67353

In [9]:
reviews.rating.isnull().sum()

0

In [10]:
reviews.rating.mean()

7.3158776939942074

In [11]:
reviews.rating.describe()

count    863866.000000
mean          7.315878
std           1.853831
min           0.000000
25%           6.000000
50%           8.000000
75%           9.000000
max          10.000000
Name: rating, dtype: float64

In [12]:
# Use your findings to match each variable to the correct statement in the dictionary


dict_sol1 = {
'The number of movies in the dataset': 35479,
'The number of ratings in the dataset': 863866,
'The number of different genres': 28,
'The number of unique users in the dataset': 67353,
'The number missing ratings in the reviews dataset': 0,
'The average rating given across all ratings': 7.3,
'The minimum rating given across all ratings': 0,
'The maximum rating given across all ratings': 10
}

# Originally, I had this to check your solution, but the 
# links are live and updating.  That didn't end up being
# a great idea

#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [13]:
# Pull the date from the title and create new column

# here's how Josh is solving this:
# He will be looking for this last parenthesis ")" as it is enclosing the date
# If the parenthesis exists, he just pulls the date from this, that is [-5:-1], otherwise he fills with nan

# He creates a lambda function he will be using it on a different column he creates later
create_date = lambda val: val[-5:-1] if val[-1] == ')' else np.nan

# Here the new column
movies['date'] = movies['movie'].apply(create_date)
movies.head()

,movie_id,movie,genre,date
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895
4,0000091,Le manoir du diable (1896),Short|Horror,1896


In [14]:
# Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)

# Josh will be creating a function to work on the date column
def add_movie_year(val):
    if val[:2] == yr:
        return 1
    else:
        return 0

# in order to have the above function working he needs to specify the yr variable

for yr in ['18', '19', '20']: # Here you have the centuries to check the yr variable above
    movies[str(yr) + "00's"] = movies['date'].apply(add_movie_year) # create column and give value

movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0


In [15]:
# Josh testing the find() function!
print(movies.genre[0].find('Short')) # the 12th index becacuse there exists the Short genre in that genre string
print(movies.genre[0].find('Horror')) # I get -1 because "Horror" does not exist in my that genre string

12
-1


In [16]:
# Dummy column the genre with 1's and 0's

# Josh will be using the find() method that it gives us the index of the first place of that genre
def split_genres(val):
    try:
        if val.find(g) > -1:      # if I get more than -1 means my genre exist in that genre string
            return 1              # we know it exists in that genre string
        else:
            return 0
    except AttributeError:
        return 0

In [17]:
# we have created the genres set above we will be using here
print(genres)

# we will be looping in the movies['genre'] column to create the columns based on what we have in the genres set
for g in genres:
    movies[g] = movies['genre'].apply(split_genres) # thanks to the split_genre function created above, we fill 0s and 1s

{'Reality-TV', 'News', 'Adult', 'Crime', 'Horror', 'Animation', 'Romance', 'Drama', 'Comedy', 'Game-Show', 'Thriller', 'Talk-Show', 'Musical', 'Fantasy', 'Western', 'War', 'Family', 'Action', 'Documentary', 'Adventure', 'Sci-Fi', 'Biography', 'Music', 'Short', 'Sport', 'Mystery', 'History', 'Film-Noir'}


In [18]:
# check out it works
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,Reality-TV,News,Adult,...,Documentary,Adventure,Sci-Fi,Biography,Music,Short,Sport,Mystery,History,Film-Noir
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [19]:
movies.iloc[1]

movie_id                                   0000010
movie          La sortie des usines Lumière (1895)
genre                            Documentary|Short
date                                          1895
1800's                                           1
1900's                                           0
2000's                                           0
Reality-TV                                       0
News                                             0
Adult                                            0
Crime                                            0
Horror                                           0
Animation                                        0
Romance                                          0
Drama                                            0
Comedy                                           0
Game-Show                                        0
Thriller                                         0
Talk-Show                                        0
Musical                        

In [20]:
# Now we will be working on the reviews dataframe
import datetime

change_timestamp = lambda val: datetime.datetime.fromtimestamp(int(val)).strftime('%Y-%m-%d %H:%M:%S')

# we apply this function to the timestamp column and saving into a new column namely date_time
reviews['date_time'] = reviews['timestamp'].apply(change_timestamp)

In [21]:
reviews.head()

,user_id,movie_id,rating,timestamp,date_time
0,1,0114508,8,1381006850,2013-10-05 21:00:50
1,2,0208092,5,1586466072,2020-04-09 21:01:12
2,2,0358273,9,1579057827,2020-01-15 03:10:27
3,2,10039344,5,1578603053,2020-01-09 20:50:53
4,2,6751668,9,1578955697,2020-01-13 22:48:17


In [ ]:
reviews_new, movies_new = t.show_clean_dataframes()